In [1]:
import pandas as pd

In [2]:
df_produkcja = pd.read_csv('electricity_production_entsoe_all.csv')
df_qq = pd.read_csv('qq_data_for_location.csv')
df_maxProdukcja = pd.read_csv('installed_capacity_entsoe_all.csv')

In [3]:
df_produkcja = df_produkcja[['date', 'solar']]
df_produkcja['date'] = pd.to_datetime(df_produkcja['date'], format='%d.%m.%Y %H:%M', errors='coerce')
df_produkcja.set_index('date', inplace=True)
# Grupowanie po godzinie i obliczanie średniej dla każdej godziny
df_produkcja = df_produkcja.resample('D').mean()
df_produkcja.dropna(inplace=True)
df_produkcja.head()

,solar
date,
2020-04-10,338.375000
2020-04-11,362.333333
2020-04-12,318.750000
2020-04-13,141.625000
2020-04-14,201.208333


In [4]:
df_qq.head()

,Date,QQ
0,2011-01-01 00:00:00,12
1,2011-01-02 00:00:00,15
2,2011-01-03 00:00:00,14
3,2011-01-04 00:00:00,20
4,2011-01-05 00:00:00,27


In [5]:
df_maxProdukcja['solarMax'] = df_maxProdukcja['solar']
df_maxProdukcja = df_maxProdukcja[['date', 'solarMax']]
df_maxProdukcja.head()

,date,solarMax
0,2015,14.0
1,2016,77.0
2,2017,187.0
3,2018,231.0
4,2019,430.0


In [6]:
df_produkcja.dtypes

solar    float64
dtype: object

In [7]:
df_qq.dtypes

Date    object
QQ       int64
dtype: object

In [8]:

df_qq['datetime'] = pd.to_datetime(df_qq['Date'], errors='coerce')
df_qq['year'] = df_qq['datetime'].dt.year
df_qq.set_index('datetime', inplace=True)

In [9]:
# Połącz dwa zestawy danych na podstawie indeksu
df_combined = df_produkcja.join(df_qq, how='inner') 
df_combined.reset_index(inplace=True)


In [10]:
df_combined.head()

,index,solar,Date,QQ,year
0,2020-04-10,338.375000,2020-04-10 00:00:00,237,2020
1,2020-04-11,362.333333,2020-04-11 00:00:00,245,2020
2,2020-04-12,318.750000,2020-04-12 00:00:00,253,2020
3,2020-04-13,141.625000,2020-04-13 00:00:00,137,2020
4,2020-04-14,201.208333,2020-04-14 00:00:00,183,2020


In [11]:
df_combined = pd.merge(df_combined, df_maxProdukcja[['date', 'solarMax']],  left_on='year', right_on='date', how='left')

df_combined['efficiency'] = df_combined['solar'] / df_combined['solarMax']


In [12]:
df_combined.head()

,index,solar,Date,QQ,year,date,solarMax,efficiency
0,2020-04-10,338.375000,2020-04-10 00:00:00,237,2020,2020,1310.0,0.258302
1,2020-04-11,362.333333,2020-04-11 00:00:00,245,2020,2020,1310.0,0.276590
2,2020-04-12,318.750000,2020-04-12 00:00:00,253,2020,2020,1310.0,0.243321
3,2020-04-13,141.625000,2020-04-13 00:00:00,137,2020,2020,1310.0,0.108111
4,2020-04-14,201.208333,2020-04-14 00:00:00,183,2020,2020,1310.0,0.153594


In [13]:
df_combined.reset_index(inplace=True) 

# df_combined.head()

In [14]:
print(df_combined.columns)

Index(['level_0', 'index', 'solar', 'Date', 'QQ', 'year', 'date', 'solarMax',
       'efficiency'],
      dtype='object')


In [15]:
df_combined.head()

,level_0,index,solar,Date,QQ,year,date,solarMax,efficiency
0,0,2020-04-10,338.375000,2020-04-10 00:00:00,237,2020,2020,1310.0,0.258302
1,1,2020-04-11,362.333333,2020-04-11 00:00:00,245,2020,2020,1310.0,0.276590
2,2,2020-04-12,318.750000,2020-04-12 00:00:00,253,2020,2020,1310.0,0.243321
3,3,2020-04-13,141.625000,2020-04-13 00:00:00,137,2020,2020,1310.0,0.108111
4,4,2020-04-14,201.208333,2020-04-14 00:00:00,183,2020,2020,1310.0,0.153594


In [16]:
df_combined = df_combined[['index', 'QQ', 'efficiency']]
df_combined.max()

index         2024-06-30 00:00:00
QQ                            462
efficiency               0.392525
dtype: object

In [17]:

df_combined.head()

,index,QQ,efficiency
0,2020-04-10,237,0.258302
1,2020-04-11,245,0.276590
2,2020-04-12,253,0.243321
3,2020-04-13,137,0.108111
4,2020-04-14,183,0.153594


In [18]:
# Zapis do pliku CSV
df_combined.to_csv('dataset.csv', index=False)  # index=False, aby nie zapisywać indeksu

print("Plik CSV został zapisany jako 'dataset.csv'")

Plik CSV został zapisany jako 'dataset.csv'
